This function checks checks whether the curve is closed.

| Name | Type (Shape) | Description |
| --- | --- | --- |
| x | list | list containing x-coordinates of curve
| y | list | list containing y-coordinates of curve
| phi | list | list containing angle $ \phi $ of the tangent of the curve
| d_threshold | float | threshold distance between starting and re-intersection of the closed curve.

In [ ]:
import sys, os

# get current directory
path = os.getcwd()

# get parent directory
parent_directory = os.path.sep.join(path.split(os.path.sep)[:-3])

# add utils folder to current working path
sys.path.append(parent_directory+"/subfunctions/utils")

In [1]:
# Import numpy
import numpy as np

# Import math symbols
from math import sqrt, pi

# Function which checks if particle is in defined domain
from ipynb.fs.defs.check_location import check_location

In [1]:
# Check if curve is closed after completing one full cycle
def closed_curve(x, y, phi, d_threshold, X, Y, defined_domain):
    '''
    Checks if function is a closed, smooth curve. If this is the case, then extract the closed and smooth segment of the curve.
    
    Parameters:
        x:              list, x-coordinate of curve
        y:              list, y-coordinate of curve
        phi:            list, angle of tangent of curve
        d_threshold:    float, threshold distance between starting and re-intersection of the closed curve.
        X:              array(Ny, Nx), X-meshgrid
        Y:              array(Ny, Nx), Y-meshgrid
        defined_domain: array(Ny, Nx), meshgrid which denotes the domain where the velocity field is defined, 1 if velocity field is defined, 0 otherwise
        
    Returns:
        closed_x:       list, x-coordinate of closed curve
        closed_y:       list, y-coordinate of closed curve
        closed_phi:     list, angle of tangent of closed curve
    '''
    
    # Discard trajectories whose maximum value was not close to 2*pi as they do not re-intersect.
    if max(np.abs(phi)) < 0.95*2*pi:
        return None, None, None
    
    # compute normal of initial curve
    k = -(x[1]-x[0])/(y[1]-y[0])
    
    # define endpoints of intersection plane
    x0 = x[0]
    y0 = y[0]
    x_end1 = sqrt(d_threshold**2/(k**2+1))+x0
    y_end1 = k*(x_end1-x0)+y0
    x_end2 = -sqrt(d_threshold**2/(k**2+1))+x0
    y_end2 = k*(x_end2-x0)+y0
    
    # compute tangent vector
    t_vector = [x[1]-x[0], y[1]-y[0]]
    
    # define line
    line0 = [(x_end1, y_end1), (x_end2, y_end2)]
    
    # iterate over curve
    for i in range(1, x.shape[0]-1):
        
        # Check if particle is in a domain where velocity field is defined
        # If outside domain then None, None, None is returned and integration will stop.
        if check_location(X, Y, defined_domain, np.array([x[i], y[i]]), True)[0] != "IN":
            return None, None, None
        
        # check if vector is aligned with initial tangent vector
        if np.sign((x[i+1]-x[i])*t_vector[0]+(y[i+1]-y[i])*t_vector[1]) >= 0:
            
            # check if curve re-intersects and if angle close to 2*pi (this guarantees that curve is smooth)
            line1 = [(x[i], y[i]), (x[i+1], y[i+1])]
            if intersects(line0, line1) and abs((abs(phi[i])-2*pi)%(2*pi))<=0.025:
                closed_x = x[:i+1]
                closed_y = y[:i+1]
                closed_phi = phi[:i+1]
                return closed_x, closed_y, closed_phi
        
    return None, None, None

In [3]:
# assumes line segments are stored in the format [(x0,y0),(x1,y1)]
# check if two curves intersect
def intersects(s0,s1):
    '''
    Checks if two curves intersect
    '''
    
    dx0 = s0[1][0]-s0[0][0]
    dx1 = s1[1][0]-s1[0][0]
    dy0 = s0[1][1]-s0[0][1]
    dy1 = s1[1][1]-s1[0][1]
    p0 = dy1*(s1[1][0]-s0[0][0]) - dx1*(s1[1][1]-s0[0][1])
    p1 = dy1*(s1[1][0]-s0[1][0]) - dx1*(s1[1][1]-s0[1][1])
    p2 = dy0*(s0[1][0]-s1[0][0]) - dx0*(s0[1][1]-s1[0][1])
    p3 = dy0*(s0[1][0]-s1[1][0]) - dx0*(s0[1][1]-s1[1][1])
    return (p0*p1<=0) & (p2*p3<=0)